# Data Exploration & Cleaning Notebook

**Project**: Heart Disease Prediction

**Date**: 2026-01-20

The goals of this analysis are:

1) Perform a thorough analysis of the 

Column Descriptions:

| Var. Name |  Role | Type | Units | Notes |
|------|------|------|-------|-------|
|  Age |  Predictor  |  Int  | years | Age of the patient |
|  Sex |  Predictor  |  Categorical | N/A | Male/Female  |
| Chest Pain | Predictor | Categorical | N/A | Chest pain type |
| Rest BP | Predictor | Integer | mmHg | Resting blood pressure (on admission to the hospital) |
| Chol | Predictor | Integer | mg/dl | Serum cholesterol |
| FBS | Predictor | Categorical | N/A | Fasting blood sugar > 120 mg/dl |
| Rest ECG | Predictor | Categorical | N/A | Resting electrocardiographic results |
| Max HR | Predictor | Integer | bpm | 	Maximum heart rate achieved |
| Ex Angina | Predictor | Categorical | N/A | exercise-induced angina |
| Oldpeak | Predictor | Integer |  mm | ST depression induced by exercise relative to rest |
| Slope | Predictor | Categorical | N/A | The slope of the peak exercise ST segment |
| Ca | Predictor |  Integer | N/A | Number of major vessels (0-3) colored by fluoroscopy |
| Thal | Predictor | Categorical | N/A | normal; fixed defect; reversible defect | 
| CVD Class | Target | Integer | N/A | Diagnosis of heart disease (0-4) |

Additional Information:

Chest Pain Type: 
 typical angina, atypical angina, non-anginal, asymptomatic

Rest ECG: 
normal, stt abnormality, lv hypertrophy

Thal: 
normal; fixed defect; reversible defect

CVD Class
0=no heart disease; 1,2,3,4 = stages of heart disease 

Creators:

Hungarian Institute of Cardiology. Budapest: Andras Janosi, M.D.
University Hospital, Zurich, Switzerland: William Steinbrunn, M.D.
University Hospital, Basel, Switzerland: Matthias Pfisterer, M.D.
V.A. Medical Center, Long Beach and Cleveland Clinic Foundation: Robert Detrano, M.D., Ph.D.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy.cluster.hierarchy import dendrogram, linkage
from scipy.stats import chi2_contingency

from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', None)

# Set plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

In [ ]:
# Load the datasets
files = ['cleveland.data', 'hungarian.data', 'switzerland.data', 'va.data']
directory, file_prefix = 'data', 'processed'

# Anynomous function to create the relative file paths 
make_filepath = lambda directory, file_prefix, filename: f"./{directory}/{file_prefix}.{filename}"
filepaths = [make_filepath(directory, file_prefix, f) for f in files]

# Defining the column names 
cols = ["Age", "Sex", "Chest Pain", "Rest BP", "Chol", "FBS", "Rest ECG", "Max HR", "Ex Angina", "Oldpeak", "Slope", "Ca", "Thal", "CVD Class"]
dataset_names = ['Cleveland', 'Hungarian', 'Switzerland', 'VA Long Beach']

dfs = [pd.read_csv(f, sep=",") for f in filepaths]

# Assign the same column name to each dataframe
for df in dfs:
    df.columns = cols

for i, df in enumerate(dfs):
    df['Dataset'] = dataset_names[i]

df = pd.concat(dfs, ignore_index=True)

df.head()

In [ ]:
print(f"Dataset shape: {df.shape}")
print(f"Rows: {df.shape[0]:,}, Columns: {df.shape[1]}")

In [ ]:
# Data types
print("\n=== Data Types ===")
print(df.dtypes.value_counts())
print("\nDetailed data types:")
print(df.dtypes)

# Basic info
print("\n=== Dataset Info ===")
df.info()

In [ ]:
# Numeric columns
numeric_cols = df.select_dtypes(include="number").columns.tolist()

df[numeric_cols].describe().round(2)

In [ ]:
# Number of unique values in the categorical columns
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

unique_count = df[categorical_cols].nunique()

unique_count

In [ ]:
# Replacing all the question marks with NaN values
df = df.replace("?", np.nan)

missing_data = pd.DataFrame({
    'Missing_Count': df.isnull().sum(),
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).round(2)
})

missing_data = missing_data[missing_data['Missing_Count'] > 0].sort_values('Missing_Percentage', ascending=False)

missing_data

Here we face a dillema with the missing values. I'll break them down into two categories: high missingness (>30% missing) and low missingness (<10% missing).

I want to use the best approach possible for dealing with these features. For example, might it be better to perform imputation on the columns that have low missingness (<10% of total rows)? For those that have a significant number of missing values (Ca, Thal, Slope), perhaps performing some kind of unsupervised learning method to approximate the most likely value using other features would make sense? Perhaps simply dropping the features would be sufficient.

When looking at the description of the high missingness features, we see that they are all cardiac stress test results that require specialized equipment or procedures, which explains why a large quantity are missing. Because they are medically significant indicators of cardiac health, simply dropping these predictors will likely not improve the model's performance.

- **ca**: number of major vessels (0-3) colored by flourosopy

- **thal**: 3 = normal; 6 = fixed defect; 7 = reversable defect

- **slope**: the slope of the peak exercise ST segment 
    -- Value 1: upsloping 
    -- Value 2: flat 
    -- Value 3: downsloping

For the other values, using simple imputation is likely to be fine. 

In [ ]:
missing_cols = missing_data.index 

# Create binary missingness indicators
missing_matrix = df[missing_cols].isnull().astype(int)
missing_matrix.columns = [f"{col}_missing" for col in missing_cols]

# Calculate co-occurrence (correlation of missingness)
cooccurrence = missing_matrix.corr()

cooccurrence.head(3).round(2)

In [ ]:
# Plotting the correlation of missing features
fig, ax = plt.subplots(ncols=2, figsize=(12, 5))
mask = np.triu(np.ones_like(cooccurrence, dtype=bool), k=1)
sns.heatmap(cooccurrence, annot=True, fmt='.2f', cmap='vlag',
            center=0, square=True, mask=mask, cbar_kws={"shrink": 0.8}, ax=ax[0])
ax[0].set_title('Missingness Co-occurrence Matrix', )

# Cluster features by missingness patterns
linkage_matrix = linkage(missing_matrix.T, method='ward')

dendrogram(linkage_matrix, labels=[col.replace('_missing', '') 
                                    for col in missing_matrix.columns],
           leaf_rotation=50, leaf_font_size=10, ax=ax[1])
ax[1].set_title('Hierarchical Clustering of Features by Missingness Patterns')
ax[1].set_xlabel('Features')
ax[1].set_ylabel('Distance')


plt.tight_layout()
plt.show()

Looking at the results from this analysis, it is clear that the patterns in missingness are not a coincidence. The dendogram and correlation matrix confirm that Slope, Ca, and Thal have a high co-occurence rate and are clustered together. Nonetheless, other features that have lower missing incidence seem to have even greater correlation and smaller distances from each other than the features that are missing a significant amount of values.

This may indicate that certain regions perform these tests together, and others may forgo them entirely. It's a nice proxy visualization for the four different datasets that are merged together, indicating the population differences that exist between them.

In [ ]:
# 
high_miss_features = ['Ca', 'Thal', 'Slope']
target_col = 'CVD Class'

# Create visualization
fig, axes = plt.subplots(ncols=3, figsize=(15, 5))

for idx, feature in enumerate(high_miss_features):
    if idx < len(high_miss_features):
        is_missing = df[feature].isnull()
        cross_tab = pd.crosstab(df[target_col], is_missing, normalize='columns') * 100
        
        cross_tab.plot(kind='bar', ax=axes[idx], color=['#2ecc71', '#e74c3c'])
        axes[idx].set_title(f'{feature} Missingness vs {target_col}')
        axes[idx].set_xlabel(target_col)
        axes[idx].set_ylabel('Percentage')
        axes[idx].legend(['Present', 'Missing'], loc='best')
        axes[idx].set_xticklabels(axes[idx].get_xticklabels(), rotation=45)
        
        # Chi-square test
        contingency = pd.crosstab(df[target_col], is_missing)
        chi2, p_value, dof, expected = chi2_contingency(contingency)
        axes[idx].text(0.02, 0.98, f'p-value: {p_value:.3f}', 
                        transform=axes[idx].transAxes, 
                        verticalalignment='top',
                        bbox=dict(boxstyle='round', facecolor='skyblue', alpha=0.5))

The results above suggest that the individuals who have missing values and those who have the values present across the top three missing predictors (Ca, Thal, Slope), come from a different population. This makes sense as the dataset is composed of individuals from different regions of the world (United States, Hungary, Switzerland) which may have very different lifestyles and diets, along with wealth (which may explain why )

As a result of this analysis we can confidently say there is a pattern that exists within the missing data, and therefore it is *Missing Not At Random (MNAR)*.

## Implications

- Missingness is structured by data source (different hospitals/regions).
- The populations are fundamentally different (lifestyle, diet, wealth, healtcare system).
- Missing tests likely reflect different diagnostic protocols rather than random omission.

## Verdict

Perhaps imputation should occur on a dataset-by-dataset basis. Since each dataset comes from a different population, an analysis of the different datasets; observing and quantifying their similarities and differences would be valuable. The end goal for this project is to build a model that is good at predicting what category of cardiovascular disease someone belongs to (0-4), where 0=no heart disease; 1,2,3,4 = stages of heart disease.

The hypothesis is that there are common factors regardless of the population we're studying that will influence the development, and as a result, the stage of heart disease in an individual. In this part of the project however, I want to quantify and explore the differences between the different datasets to gain a deeper understanding of the different populations.

This will be done in the next section below

# Dataset (Population) Differences

In [ ]:
overview_stats = []
for name, df in zip(dataset_names, dfs):
    total_cells = df.shape[0] * df.shape[1]
    df.replace('?', np.nan, inplace=True) # Important step as they won't be count as missing if this isn't done 
    missing_cells = df.isnull().sum().sum()
    complete_cases = (~df.isnull().any(axis=1)).sum()
    
    overview_stats.append({
        'Dataset': name,
        'N_Observations': df.shape[0],
        'N_Features': df.shape[1],
        'Complete_Cases': complete_cases,
        'Complete_Cases_%': (complete_cases / df.shape[0] * 100),
        'Missing_Cells': missing_cells,
        'Missing_%': (missing_cells / total_cells * 100)
    })

overview_df = pd.DataFrame(overview_stats)

overview_df.round(3)

In [ ]:
# Visualize dataset sizes and completeness
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Sample sizes
axes[0].bar(dataset_names, [df.shape[0] for df in dfs], 
            color=['#3498db', '#e74c3c', '#2ecc71', '#f39c12'])
axes[0].set_title('Sample Size by Dataset')
axes[0].set_ylabel('Number of Observations')
axes[0].set_xlabel('Dataset')
for i, (name, df) in enumerate(zip(dataset_names, dfs)):
    axes[0].text(i, df.shape[0] + 5, str(df.shape[0]), 
                ha='center', fontweight='bold')

# Completeness
completeness = [overview_stats[i]['Complete_Cases_%'] for i in range(len(dfs))]
axes[1].bar(dataset_names, completeness, 
            color=['#3498db', '#e74c3c', '#2ecc71', '#f39c12'])
axes[1].set_title('Data Completeness by Dataset')
axes[1].set_ylabel('% Complete Cases')
axes[1].set_xlabel('Dataset')
for i, val in enumerate(completeness):
    axes[1].text(i, val + 1, f'{val:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

In [ ]:
# Calculate missingness percentage for each feature in each dataset
missingness_comparison = pd.DataFrame()
for name, df in zip(dataset_names, dfs):
    miss_pct = (df.isnull().sum() / len(df) * 100).round(2)
    missingness_comparison[name] = miss_pct

missingness_comparison = missingness_comparison.drop('Dataset', errors='ignore')
missingness_comparison = missingness_comparison[missingness_comparison.sum(axis=1) > 0]

print("\nMissingness % by Feature and Dataset:")
missingness_comparison

In [ ]:
# Visualize missingness patterns
fig, ax = plt.subplots(ncols=2, figsize=(14, 5))

missingness_comparison.T.plot(kind='barh', ax=ax[0])
ax[0].set_title('Feature Missingness Across Datasets')
ax[0].set_ylabel('Dataset')
ax[0].set_xlabel('Missing (%)')
ax[0].legend(title='Features', bbox_to_anchor=(1.05, 1), loc='upper left')
ax[0].grid(axis='y', alpha=0.3)

# Heatmap version
sns.heatmap(missingness_comparison, annot=True, fmt='.1f', cmap='YlOrRd', 
            cbar_kws={'label': 'Missing %'}, ax=ax[1])
ax[1].set_title('Missingness Heatmap: Features vs Datasets')
ax[1].set_xlabel('Dataset')
ax[1].set_ylabel('Feature')

plt.tight_layout()
plt.show()

In [ ]:
# Cross-tabulation
cvd_crosstab = pd.crosstab(df['Dataset'], df['CVD Class'], 
                            margins=True, margins_name='Total')
print("\nAbsolute counts:")
print(cvd_crosstab)

# Percentage distribution (excluding margins)
cvd_pct = pd.crosstab(df['Dataset'], df['CVD Class'], 
                      normalize='index') * 100
print("\nPercentage distribution:")
print(cvd_pct.round(2))

# Chi-square test for independence
contingency = pd.crosstab(df['Dataset'], df['CVD Class'])
chi2, p_value, dof, expected = chi2_contingency(contingency)

print(f"\nChi-square test for independence:")
print(f"Chi-square statistic: {chi2:.4f}")
print(f"P-value: {p_value:.4e}")
print(f"Degrees of freedom: {dof}")

In [ ]:
# Visualizing the CVD class distribution
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Stacked bar chart
cvd_pct.plot(kind='bar', stacked=True, ax=axes[0], color=['#2ecc71', '#f39c12', '#e67e22', '#e74c3c', '#9b59b6'])
axes[0].set_title('CVD Class Distribution by Dataset (Stacked %)')
axes[0].set_xlabel('Dataset')
axes[0].set_ylabel('Percentage')
axes[0].legend(title='CVD Class', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[0].set_xticklabels(axes[0].get_xticklabels(), rotation=45, ha='right')

# Grouped bar chart
cvd_pct.plot(kind='bar', ax=axes[1], color=['#2ecc71', '#f39c12', '#e67e22', '#e74c3c', '#9b59b6'])
axes[1].set_title('CVD Class Distribution by Dataset (Grouped %)')
axes[1].set_xlabel('Dataset')
axes[1].set_ylabel('Percentage')
axes[1].legend(title='CVD Class', bbox_to_anchor=(1.05, 1), loc='upper left')
axes[1].set_xticklabels(axes[1].get_xticklabels(), rotation=45, ha='right')
axes[1].grid(axis='y', alpha=0.3)

plt.tight_layout()
plt.show()